Homework 3: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

In [2]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [4]:
user_question = "I just discovered the course. Can I still join it?"

In [5]:
question_vector = embedding_model.encode(user_question)

array([ 7.82226175e-02, -4.01311480e-02,  3.86136174e-02, -1.78979419e-04,
        8.92346054e-02, -5.04591353e-02, -1.05027072e-02,  3.71056125e-02,
       -4.18713838e-02,  3.48085277e-02, -1.20701995e-02, -2.36942172e-02,
        3.87899689e-02,  1.60988104e-02,  3.50747332e-02,  3.04757687e-03,
        5.79672195e-02, -4.10627648e-02, -3.41552906e-02, -2.56396793e-02,
       -3.55264321e-02,  1.42908283e-02, -1.62800662e-02,  3.21446136e-02,
       -4.66897637e-02,  7.89186060e-02,  4.90160808e-02,  1.56760961e-02,
       -1.69110000e-02,  2.26482209e-02,  5.60205802e-02, -3.98361497e-02,
        6.77409470e-02, -1.20210163e-02,  1.12619717e-03, -1.94394737e-02,
       -2.65951362e-02,  1.06178075e-02,  1.69687793e-02,  1.13488315e-02,
       -2.97062900e-02,  5.25258556e-02, -1.41453119e-02,  4.61699925e-02,
        1.17066409e-02, -2.38052960e-02, -6.32557943e-02, -1.92041676e-02,
       -7.10593350e-03,  3.24167535e-02,  2.49618571e-02, -5.27498079e-03,
        2.01149583e-02, -

In [6]:
len(question_vector)

768

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [10]:
len(documents)

948

In [12]:
filtered_docs = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(filtered_docs)

375

In [13]:
import numpy as np

In [17]:
embeddings = []
for doc in filtered_docs:
    qa_text = doc['question'] + ' ' + doc['text']
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

len(embeddings)

375

In [20]:
X = np.array(embeddings)
X.shape

(375, 768)

In [21]:
scores = X.dot(question_vector)
scores

array([ 2.89217353e-01,  4.35050905e-01,  2.20572829e-01,  1.28085971e-01,
        3.28754663e-01,  4.58999872e-01,  4.48930979e-01,  2.70170629e-01,
        2.13975087e-01,  3.39745581e-01,  1.63339287e-01,  3.15917969e-01,
        1.53619647e-02,  4.14240628e-01,  6.50657415e-01,  4.12786543e-01,
        2.02390388e-01,  4.71707642e-01,  3.74821603e-01,  3.53319407e-01,
        4.50558662e-01,  1.85430780e-01,  3.45527232e-01,  8.22948888e-02,
       -2.19076872e-02, -4.64249700e-02,  8.60691220e-02,  9.87197086e-02,
       -2.14188099e-02,  1.28752533e-02,  2.54871063e-02,  3.66865620e-02,
        4.56339382e-02, -2.05278154e-02,  1.82662141e-02,  3.39788869e-02,
        1.66501384e-03,  1.03211161e-02,  3.40170935e-02, -2.65231486e-02,
       -4.11909074e-02, -4.97983694e-02, -6.20346591e-02, -7.63554219e-03,
        4.61618155e-02,  6.95945695e-03,  4.77613434e-02, -9.97793861e-04,
       -5.60125113e-02, -5.96207306e-02,  2.05209255e-01, -1.16968535e-01,
       -2.34159324e-02, -

In [22]:
best_score = np.max(scores)
best_score

0.6506574

In [23]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(question_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [24]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
len(ground_truth)

1830

In [25]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [32]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        q_vec = embedding_model.encode(q['question'])
        doc_id = q['document']
        results = search_function(q_vec)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [34]:
search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)

evaluate(ground_truth, search_engine.search)

100%|██████████| 1830/1830 [00:49<00:00, 37.33it/s]


{'hit_rate': 0.9579234972677596}

In [37]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [40]:
# Adding the 'question_text_vector' key to each dictionary in filtered_docs
for doc, embedding in zip(filtered_docs, embeddings):
    doc['question_text_vector'] = embedding

In [42]:
for doc in tqdm(filtered_docs):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:01<00:00, 245.29it/s]


In [43]:
def elastic_search_knn(vector):
    knn = {
        "field": 'question_text_vector',
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [45]:
elastic_search_knn(question_vector)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [46]:
evaluate(ground_truth, elastic_search_knn)

100%|██████████| 1830/1830 [00:39<00:00, 46.71it/s]


{'hit_rate': 0.9398907103825137}